In [1]:
import numpy as np
import math
from random import randint

Реализация крестового метода:

In [9]:
def crest(A, eps):
    m, n = A.shape
    I = np.ones(m, dtype=int)
    J = np.ones(n, dtype=int)
    r = 1
    
    j = randint(0, n)
    
    
    
    i = np.argmax(abs(A[:,j]))
    U = A[:,j].reshape(m, 1) / math.sqrt(abs(A[i][j]))
    V = A[i,:].reshape(1,n) * math.sqrt(abs(A[i][j])) / A[i][j]
    
    I[i] = 0
    J[j] = 0
    
    j = np.argmax(abs(A[i,:] * J))
    
    norm_f = np.linalg.norm(U) * np.linalg.norm(V)
    
    
    while True:
        r += 1
        A_vawe_j = A[:,j] - U @ (V[:,j])
        
        i = np.argmax(abs(A_vawe_j) * I)
        
        A_vawe_i = A[i,:] - U[i] @ V
        
        if (abs(A_vawe_i[j]) * math.sqrt((m-r) * (n-r)) < eps * norm_f) or r==min(m,n):
            r -= 1
            break
        
        u = A_vawe_j / math.sqrt(abs(A_vawe_j[i]))
        v = A_vawe_i * math.sqrt(abs(A_vawe_j[i])) / A_vawe_j[i]
        
        s = np.dot(U.T @ u, V @ v.T)
        
        norm_f = math.sqrt(norm_f ** 2 + (np.linalg.norm(u) * np.linalg.norm(v))**2 + s * 2)
        
        U = np.hstack((U, u.reshape(m,1)))
        V = np.vstack((V, v))

        I[i] = 0
        J[j] = 0
        
        j = np.argmax(abs(A_vawe_i) * J)
    
    return (U, V, r)

### Матрица гильберта

In [3]:
def A(i, j):
    return 1.0/(i+j+1)

In [4]:
M = 1024 # number of rows
N = 1500 # number of columns
x = np.linspace(1,N,N) # grid for drawings
r = 20 # target rank
# fill the matrix Ma
Ma=np.fromfunction(A, [M, N])

In [6]:
U, V, r = crest(Ma, 1e-5)

In [7]:
r

11

In [8]:
np.linalg.norm(Ma - U @ V)

7.313595850569189e-05

In [9]:
np.linalg.matrix_rank(U @ V)

11

Задаем функцию вычисляющую элементы матрицы

In [10]:
def A1(i, j):
    return ((i+1) ** (1/3)) * ((j+1) ** (-1/3)) + ((i+1) ** (-1/3)) * ((j+1) ** (1/3))

Задаем матрицу

In [11]:
M = 8192 # number of rows
N = 8192 # number of columns
x = np.linspace(1,N,N) # grid for drawings
r = 2 # target rank
# fill the matrix Ma
Ma1=np.fromfunction(A1, [M, N])
Ma1.shape

(8192, 8192)

In [12]:
from sklearn.utils.extmath import randomized_svd as rsvd
import time

Сравнение времени работы

In [16]:
start = time.time()
U, s, V = np.linalg.svd(Ma1)
print("Classical ", time.time() - start, " sec")

Classical  166.96399116516113  sec


In [16]:
start = time.time()
Ur, sr, Vr = rsvd(Ma1, n_components=r, random_state=np.random.randint(1))
print("Randomized ", time.time() - start, " sec")

Randomized  1.7319884300231934  sec


In [15]:
start = time.time()
U_crest, V_crest, r_crest = crest(Ma1, 1e-10)
print("Crest ", time.time() - start, " sec")

Crest  0.004000186920166016  sec


зависимость ранга относительно критерия остановки для матрицы $A(i, j) = i^{1/3}j^{-1/3}+i^{-1/3}j^{1/3}$

In [18]:
epsilons = [1e-3, 1e-4, 1e-5]
for eps in epsilons:
    U_c, V_c, r_c = crest(M, N, Ma1, eps)
    print(r_c)

2
2
2


Реализация усеченного SVD с адаптацией ранга по значению целевой точности прибилжения $\varepsilon$

In [10]:
def truncated_svd(A, eps):
    U, V, r = crest(A, eps)
    
    U1, s1, V1 = np.linalg.svd(U, full_matrices=False)
    U2, s2, V2 = np.linalg.svd(V, full_matrices=False)
    G = np.diag(s1) @ V1 @ U2 @ np.diag(s2)

    Ug, sg, Vg = np.linalg.svd(G)
    
    return(U1 @ Ug, sg, Vg @ V2)

In [8]:
U_t, s_t, V_t = truncated_svd(Ma, 1e-5)

In [9]:
np.linalg.norm(Ma - U_t @ np.diag(s_t) @ V_t)

6.202931593363492e-07

Сжатие черно-белого изображения

In [2]:
from PIL import Image

In [16]:
im = Image.open('jaguar.jpg')

In [17]:
im  = im.convert('L') # делаем черно-белым
pixels = np.asarray(im)

In [18]:
for i in range(5, 0, -1):
    pix_U_t, pix_s_t, pix_V_t = truncated_svd(pixels, 10 ** (-i))
    print("rank = ", pix_U_t.shape[1])
    truncated_pixels = pix_U_t @ np.diag(pix_s_t) @ pix_V_t
    result = truncated_pixels.astype(np.uint8)
    data = Image.fromarray(result, 'L')
    data.save(f'compressed_imgs/jaguar_10-{i}.jpg')

rank =  1160
rank =  1160
rank =  1160
rank =  990
rank =  948


### Real Rank

In [19]:
np.linalg.matrix_rank(pixels)

1160